In [1]:
import sys
import os
from pathlib import Path
import pickle
import pprint

# --- SETUP: Thêm 'src' vào Python path để tìm được module 'vctp' ---
# Lấy đường dẫn tới thư mục gốc của project (VisualCoT-NLE)
project_root = Path(".").resolve().parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# --- Bây giờ có thể import các module của project ---
from vctp.data.preprocess.object_similarity.processor import Processor
from vctp.data.preprocess.object_similarity.metrics import AnswerSimilarityMetric
from vctp.data.preprocess.object_similarity.similarity_builder import ObjectSimilarityBuilder

# --- 1. ĐỊNH NGHĨA CÁC ĐƯỜNG DẪN CẦN THIẾT ---
project_root = Path(".").resolve().parent

# data nằm NGOÀI project_root, cùng cấp với VisualCoT-NLE
workspace_root = project_root.parent  # Lên 1 cấp: /home/research/workspace/
data_root = workspace_root / "data"

sg_path = data_root / "raw" / "scene-graph"
annotations_path = data_root / "raw" / "vivqa-x" / "annotations"
output_dir = project_root / "notebooks" / "test_output"
output_dir.mkdir(exist_ok=True)
output_file = output_dir / "test_similarity.pkl"

print(f"✅ Setup complete. Using Scene Graphs from: {sg_path}")
print("-" * 50)


# --- 2. INPUT 1: LẤY 1 MẪU DỮ LIỆU TỪ VIVQA-X ---
processor = Processor(annotations_path)
all_questions, all_answers, all_rationales = processor.load_split("train")

print(all_answers)
# Lấy mẫu đầu tiên để test
sample_key = list(all_questions.keys())[0]
sample_question = {sample_key: all_questions[sample_key]}
sample_answer = {sample_key: all_answers[sample_key]}
image_id = int(sample_key.split("<->")[0])

print("INPUT 1: Dữ liệu câu hỏi & trả lời")
print(f"  - Key: {sample_key}")
print(f"  - Image ID: {image_id}")
print(f"  - Question: '{sample_question[sample_key]}'")
print(f"  - Answer: {sample_answer[sample_key]}")
print("-" * 50)


# --- 3. INPUT 2: LẤY CÁC OBJECTS TỪ SCENE GRAPH ---
metric = AnswerSimilarityMetric()
builder = ObjectSimilarityBuilder(
    sg_dir=sg_path / "scene_graph_coco14",
    sg_attr_dir=sg_path / "scene_graph_coco14_attr",
    metric=metric,
)

# Load các object tương ứng với image_id
objects, _ = builder.load_objects(image_id)
print("INPUT 2: Các objects từ Scene Graph")
print(f"  - Tìm thấy {len(objects)} objects cho ảnh {image_id}:")
print(f"  - {objects}")
print("-" * 50)


# --- 4. CHẠY BUILDER ĐỂ TÍNH SIMILARITY ---
print("🚀 Bắt đầu tính toán similarity...")
similarity_dict = builder.build(
    questions=sample_question,
    answers=sample_answer,
    rationales=all_rationales, # Cung cấp rationales
    output_path=output_file,
    show_progress=False  # Tắt thanh progress bar
)
print("✅ Tính toán hoàn tất!")
print("-" * 50)


# --- 5. OUTPUT 1: XEM KẾT QUẢ TRỰC TIẾP ---
print("OUTPUT 1: Dictionary kết quả trả về từ hàm build()")
pprint.pprint(similarity_dict)
print("-" * 50)


# --- 6. OUTPUT 2: ĐỌC LẠI TỪ FILE .PKL ĐÃ LƯU ---
print(f"OUTPUT 2: Đọc dữ liệu từ file đã lưu: {output_file}")
with open(output_file, "rb") as f:
    loaded_data = pickle.load(f)

print("Nội dung file .pkl:")
pprint.pprint(loaded_data)

/home/research/miniconda3/envs/visual-cot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Setup complete. Using Scene Graphs from: /home/research/workspace/data/raw/scene-graph
--------------------------------------------------
{'262146<->262146001': ['trượt tuyết'], '393223<->393223000': ['có'], '393230<->393230000': ['trượt tuyết'], '393230<->393230003': ['trượt tuyết'], '137045<->137045002': ['gấu'], '458763<->458763004': ['gỗ'], '567990<->567990002': ['dày'], '393286<->393286001': ['không'], '71<->71000': ['có'], '71<->71001': ['có'], '524375<->524375001': ['gặm cỏ'], '524377<->524377007': ['không'], '92<->92001': ['có'], '546151<->546151001': ['có'], '21826<->21826002': ['trượt tuyết'], '110<->110002': ['phô mai'], '127<->127001': ['trắng'], '142<->142000': ['chuối'], '131215<->131215001': ['có'], '144<->144002': ['hươu cao cổ'], '149<->149001': ['không'], '262308<->262308002': ['đúng'], '131245<->131245002': ['chăn gia súc'], '393394<->393394001': ['đúng'], '393394<->393394003': ['có'], '393396<->393396000': ['có'], '393396<->393396001': ['u ám'], '194<->194002': ['